In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
tokenizer = Tokenizer()

In [4]:
import string, re, nltk, os
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import Tokenizer

In [5]:
# load the doc in memory
def load_doc(filename):
    f = open(filename)
    text = f.read()
    f.close()
    return text

In [6]:
load_doc('../input/review_polarity/txt_sentoken/pos/cv000_29590.txt')


'films adapted from comic books have had plenty of success , whether they\'re about superheroes ( batman , superman , spawn ) , or geared toward kids ( casper ) or the arthouse crowd ( ghost world ) , but there\'s never really been a comic book like from hell before . \nfor starters , it was created by alan moore ( and eddie campbell ) , who brought the medium to a whole new level in the mid \'80s with a 12-part series called the watchmen . \nto say moore and campbell thoroughly researched the subject of jack the ripper would be like saying michael jackson is starting to look a little odd . \nthe book ( or " graphic novel , " if you will ) is over 500 pages long and includes nearly 30 more that consist of nothing but footnotes . \nin other words , don\'t dismiss this film because of its source . \nif you can get past the whole comic book thing , you might find another stumbling block in from hell\'s directors , albert and allen hughes . \ngetting the hughes brothers to direct this seem

In [7]:
from nltk.tokenize import word_tokenize
swords = stopwords.words('english')

In [8]:
# Function to clean the doc
def clean_doc(doc):
    tokens = word_tokenize(doc)
    tokens = [token for token in tokens if token.isalpha()]
    tokens = [token for token in tokens if token not in swords]
    tokens = [token for token in tokens if len(token) > 1]
    return tokens


In [10]:
f = open('../input/vocab.txt')
vocab = f.read().split()
f.close()
vocab;

In [11]:
# Load the file, clean the data and return

def doc_to_line(filename):
    doc = load_doc(filename)
    tokens = clean_doc(doc)
    tokens = [token for token in tokens if token in vocab]

    return ' '.join(tokens)

doc_to_line('../input/review_polarity/txt_sentoken/pos/cv000_29590.txt')

'films adapted comic books plenty success whether superheroes batman superman spawn geared toward kids casper arthouse crowd ghost world never really comic book like hell starters created alan moore eddie campbell brought medium whole new level mid series called say moore campbell thoroughly subject jack ripper would like saying michael jackson starting look little odd book graphic novel pages long includes nearly consist nothing footnotes words dismiss film source get past whole comic book thing might find another stumbling block hell directors albert allen hughes getting hughes brothers direct seems almost ludicrous casting carrot top well anything better direct film set ghetto features really violent street crime mad geniuses behind menace ii society ghetto question course whitechapel london east end filthy place whores called unfortunates starting get little nervous mysterious psychopath carving profession surgical precision first stiff turns copper peter robbie coltrane world enou

## reading all files using os

In [12]:


# process docs and load all the files from directory
def process_train(directory, is_train):
    documents = []
    for filename in os.listdir(directory):
        if is_train and filename.startswith('cv9'):
            continue
        if not is_train and not filename.startswith('cv9'):
            continue
       
        path = directory + '//' + filename
        doc = load_doc(path)
        tokens = clean_doc(doc)
        documents.append(tokens)
    return documents

In [13]:
all_pos_files = '../input/review_polarity/txt_sentoken/pos/'
all_neg_files = '../input/review_polarity/txt_sentoken/neg/'
def load_data(is_train):
    neg = process_train(all_neg_files, is_train=is_train)
    pos = process_train(all_pos_files, is_train=is_train)

    docs = neg + pos
    lables = [0 for i in range(len(neg))] + [1 for i in range(len(pos))]
    return docs, lables

In [14]:
# load training data
train_data, train_labels = load_data(True)
# load testing data
test_data, test_labels = load_data(False)

len(train_data), len(train_labels), len(test_data), len(test_labels)

(1810, 1810, 200, 200)

In [15]:
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer



In [16]:
tokenizer = create_tokenizer(train_data)

In [17]:
X_train = tokenizer.texts_to_matrix(train_data)

In [18]:
X_train.shape

(1810, 36389)

In [19]:
X_test = tokenizer.texts_to_matrix(test_data)
X_test.shape

(200, 36389)

Ae lo paaji

In [26]:
# convert data to sequences and pad
def encode_docs(tokenizer, max_length, docs):
    encoded = tokenizer.texts_to_sequences(docs)
    padded_data = pad_sequences(encoded, maxlen=max_length, padding='post')
    return padded_data

In [27]:
max_length = max([len(sent) for sent in train_data]) # max length of sentence in train data


In [43]:
# encoding

X_train = encode_docs(tokenizer, max_length, train_data)
X_test = encode_docs(tokenizer, max_length, test_data)

In [44]:
X_train.shape, X_test.shape  # less dimension in axis 1 than before

((1810, 1335), (200, 1335))

In [33]:
vocab_size = len(vocab)

In [58]:
def define_model(max_length): 
    model = tf.keras.Sequential([

        # Embedding layer
        tf.keras.layers.Embedding(vocab_size, 100),  # vocab size, embedding vector length (can be any no.), max. input length

        # feature extraction
        tf.keras.layers.Conv1D(filters=16, kernel_size=5, activation='relu'), # 16 filters, size 5
        tf.keras.layers.MaxPool1D(pool_size=2),
        tf.keras.layers.Flatten(),

        # dense layers
        tf.keras.layers.Dense(10, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    loss_fn = tf.keras.losses.BinaryCrossentropy()
    optimizer = tf.keras.optimizers.Adam()
    metrics = ['accuracy']
    model.compile(loss=loss_fn, optimizer=optimizer, metrics=metrics)
    return model

In [59]:
model = define_model(max_length)

In [60]:
# from keras.utils import plot_model
# plot_model(model=model, show_layer_activations=True)

In [61]:
hist = model.fit(X_train, np.array(train_labels), validation_data=[X_test, np.array(test_labels)], epochs=10, batch_size=18)

Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node sequential_5_1/embedding_7_1/GatherV2 defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "d:\DAI Aug-25\softwares\conda_env\dnn\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>

  File "d:\DAI Aug-25\softwares\conda_env\dnn\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "d:\DAI Aug-25\softwares\conda_env\dnn\Lib\site-packages\ipykernel\kernelapp.py", line 758, in start

  File "d:\DAI Aug-25\softwares\conda_env\dnn\Lib\site-packages\tornado\platform\asyncio.py", line 211, in start

  File "d:\DAI Aug-25\softwares\conda_env\dnn\Lib\asyncio\base_events.py", line 645, in run_forever

  File "d:\DAI Aug-25\softwares\conda_env\dnn\Lib\asyncio\base_events.py", line 1999, in _run_once

  File "d:\DAI Aug-25\softwares\conda_env\dnn\Lib\asyncio\events.py", line 88, in _run

  Fil

In [70]:
model.evaluate(X_test, np.array(test_labels), batch_size=1)

200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9100 - loss: 0.2414


[0.24135693907737732, 0.9100000262260437]

In [71]:
text1 = 'Best movie ever! It was great, I will definitely recommend it.'
text2 = 'This is a bad movie. Please don;t watch it.'

def predict_sentiment(review):
    review = clean_doc(review) # cleaning
    review = [token for token in review if token in vocab] # vocab cleaning
    review = tokenizer.texts_to_matrix([review])  # convert to matrix
    pred = model.predict(review)  # prediction

    
    if pred > 0.5:
        print('Positive', pred)
    else:
        print('Negative', 1-pred)



In [73]:
predict_sentiment(text2)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Negative [[0.6422838]]
